# Converting from Qiskit Runtime Programs

This tutorial will be a demonstation of converting your custom Qiskit Runtime Program into a Quantum Serverless `QiskitPattern`.

If you were using Qiskit Runtime Programs before, your code probably looks similar to the following example:

```python
"""A sample runtime program that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit


def prepare_circuits(backend):
    circuit = random_circuit(
        num_qubits=5, depth=4, measure=True, seed=random.randint(0, 1000)
    )
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop("iterations", 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "Hello, World!"
```


All Qiskit Runtime Programs have a `main` method which accepts `backend`, `user_messenger` and `**kwargs`. This method is not required for Quantum Serverless patterns.

Quantum Serverless handles backends, logging, and input arguments a bit differently than Qiskit Runtime:

- `backend`. For Quantum Serverless programs you are not limited to single backend for a program. You can call any
  number of backends from single program. Since `Backend.run` is deprecated, we will be using Qiskit Primitives to do our calculation.
- `user_messenger` were used in Qiskit Runtime Programs to facilitate retrieving logs from the program. Quantum Serverless does not
  require passing such an object. Instead, all contents of `stdout` (e.g. print statements, logging messages) will be provided to the
  user via the Quantum Serverless job handler.
- `**kwargs` was a variable used to capture program inputs from the user. Users should now input their arguments to the `ServerlessProvider.run` method,
  and the arguments should be retrieved within the pattern using the `get_arguments` function from Quantum Serverless.
- To save the results of a pattern, the `save_result` function should be used. It accepts a python dictionary and can be accessed via the job handler.

Let's use the guidelines above to transform the above Qiskit Runtime Program into a Quantum Serverless QiskitPattern.

```python
# migrated_pattern.py
"""A sample runtime pattern that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Sampler

from quantum_serverless import get_arguments, save_result


def prepare_circuits():
    circuit = random_circuit(
        num_qubits=5, depth=4, measure=True, seed=random.randint(0, 1000)
    )
    return transpile(circuit)


arguments = get_arguments()
iterations = arguments.get("iterations", 5)

for it in range(iterations):
    qc = prepare_circuits()
    result = Sampler.run(qc).result()
    print({"iteration": it, "dists": result.quasi_dists})

save_result({"result": "Hello, World!"})
```

Let's save this code as `./src/migrated_pattern.py` and execute it using the `QiskitPattern` class from the `quantum_serverless` package.

In [1]:
from quantum_serverless import QiskitPattern

pattern = QiskitPattern(
    title="migrated-pattern", entrypoint="migrated_pattern.py", working_dir="./src/"
)

In [2]:
from quantum_serverless import ServerlessProvider
import os

serverless = ServerlessProvider(
    username=os.environ.get("GATEWAY_USER", "user"),
    password=os.environ.get("GATEWAY_PASSWORD", "password123"),
    # token=os.environ.get("GATEWAY_TOKEN", "<TOKEN>"), # token can be used instead of user/password combination
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)
serverless

<ServerlessProvider: gateway-provider>

In [3]:
serverless.upload(pattern)

'migrated-pattern'

While Qiskit Runtime programs required users to upload their program and call it in two separate steps, the ``QiskitPattern`` class allows users to send a job for remote execution in a single step.

In [4]:
job = serverless.run("migrated-pattern", arguments={"iterations": 3})
job

<Job | 3b3d1bd1-5aa4-45a5-b2b1-fad46bd2f079>

In [5]:
job.result()

{'result': 'Hello, World!'}

In [6]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
{'iteration': 0, 'dists': [{0: 3.11944954662e-05, 1: 1.7472255688e-06, 2: 0.0023491864962565, 3: 0.0001315795832142, 4: 3.11944954662e-05, 5: 1.7472255688e-06, 6: 0.0023491864962565, 7: 0.0001315795832142, 8: 0.0003717494511414, 9: 2.08219474781e-05, 10: 0.0279956055566758, 11: 0.0015680535014343, 12: 0.0003717494511414, 13: 2.08219474781e-05, 14: 0.0279956055566758, 15: 0.0015680535014343, 16: 0.0311265176657353, 17: 0.0017434180844706, 18: 0.0031482525298452, 19: 0.0001763358321658, 20: 0.0015223386227183, 21: 0.0313475971274875, 22: 0.000153975027715, 23: 0.003170613334296, 24: 0.3709393495626852, 25: 0.0207765731205209, 26: 0.0018349433509674, 27: 0.0377846715963741, 28: 0.0181419362290861, 29: 0.3735739864541202, 30: 0.0375181945574789, 31: 0.0021014203898626}]}
{'iteration': 1, 'dists': [{0: 0.006964352275323